## 1. Simple Neural Network

Here we will build a simple neutral network, with a linear layer, a ReLU layer, and a linear layer.

- $h = w_1 x$
- $h_{ReLU} = ReLU(h)$
- $\hat{y}=w_2h_{ReLU}$

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import random

USE_CUDA = torch.cuda.is_available()

In [3]:
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)

In [4]:
N = 64
D_IN = 1000
H = 100
D_OUT = 10
NUM_EPOCHS = 500
learning_rate = 1e-6

### 1.1 Pure numpy

In [92]:
x=np.random.randn(N, D_IN)
y=np.random.randn(N, D_OUT)
w1=np.random.randn(D_IN, H)
w2=np.random.randn(H, D_OUT)

for epoch in range(NUM_EPOCHS):
    # fowward pass
    h = x.dot(w1) # (64, 1000)*(1000, 100) = (64, 100)
    h_relu = np.maximum(h, 0) # (64, 100)
    y_pred = h_relu.dot(w2) # (64, 100)*(100, 10) = (64, 10)
    
    loss = np.square(y_pred - y).sum()
    print(f"{epoch}: loss = {loss}")
    
    # calculate gradients
    
    # d loss / d y_pred = 2 * (y_pred - y)
    # d loss / d w2 = (d loss / d y_pred) * (d y_pred / d w2)
    y_pred_grad = 2*(y_pred - y) # (64,10)
    w2_grad = h_relu.T.dot(y_pred_grad) # (100, 64)*(64, 10) = (100, 10)
    
    # d loss / d h_relu =  2 * (y_pred - y) * w2
    h_relu_grad = y_pred_grad.dot(w2.T) # (64, 10)*(10, 100) = (64, 100)
    h_grad = h_relu_grad.copy() 
    h_grad[h<0] = 0 # (64, 100)
    # d loss / d h =  2 * (y_pred - y) * w2
    w1_grad = x.T.dot(h_grad) # (1000, 64) * (64, 100) = (1000, 100)
    
    # update parameters
    w1 -= learning_rate * w1_grad
    w2 -= learning_rate * w2_grad

0: loss = 24111449.222389005
1: loss = 19076708.530630156
2: loss = 19279786.257235497
3: loss = 22619014.78600002
4: loss = 27088372.012035266
5: loss = 29558662.253394634
6: loss = 26998688.822538935
7: loss = 19671584.37985573
8: loss = 11568014.60135452
9: loss = 5921807.165661035
10: loss = 2970763.928990799
11: loss = 1624117.0378756088
12: loss = 1021610.7409818026
13: loss = 733110.6716428237
14: loss = 576218.4336398772
15: loss = 477345.51577891305
16: loss = 406653.67393033847
17: loss = 351711.40464787965
18: loss = 306912.43438767747
19: loss = 269434.72992538085
20: loss = 237626.80022843674
21: loss = 210365.65284621183
22: loss = 186984.28777858437
23: loss = 166698.63525727473
24: loss = 149040.16256958485
25: loss = 133592.6333383929
26: loss = 120051.31101215395
27: loss = 108141.86333374918
28: loss = 97631.94888989479
29: loss = 88346.45534677307
30: loss = 80101.33248235229
31: loss = 72766.16415836931
32: loss = 66223.31366757283
33: loss = 60371.096216963386
34:

488: loss = 5.651874203860638e-05
489: loss = 5.4382161682401736e-05
490: loss = 5.232693847604312e-05
491: loss = 5.035029908944076e-05
492: loss = 4.844858259702501e-05
493: loss = 4.661901238419115e-05
494: loss = 4.485880319468727e-05
495: loss = 4.3165792772700186e-05
496: loss = 4.153705865155336e-05
497: loss = 3.997016154949822e-05
498: loss = 3.846257637526734e-05
499: loss = 3.70122345814878e-05


### 1.2 Pytorch Tensor

In [103]:
x=torch.randn(N, D_IN)
y=torch.randn(N, D_OUT)
w1=torch.randn(D_IN, H)
w2=torch.randn(H, D_OUT)

for epoch in range(NUM_EPOCHS):
    # fowward pass
    h = x@w1 # (64, 1000)*(1000, 100) = (64, 100)
    h_relu = F.relu(h) # (64, 100)
    y_pred = h_relu@w2 # (64, 100)*(100, 10) = (64, 10)
    
    loss = (y_pred - y).pow(2).sum().item()
    print(f"{epoch}: loss = {loss}")
    
    # calculate gradients
    
    # d loss / d y_pred = 2 * (y_pred - y)
    # d loss / d w2 = (d loss / d y_pred) * (d y_pred / d w2)
    y_pred_grad = 2*(y_pred - y) # (64,10)
    w2_grad = h_relu.T@(y_pred_grad) # (100, 64)*(64, 10) = (100, 10)
    
    # d loss / d h_relu =  2 * (y_pred - y) * w2
    h_relu_grad = y_pred_grad@w2.T # (64, 10)*(10, 100) = (64, 100)
    h_grad = h_relu_grad.clone() 
    h_grad[h<0] = 0 # (64, 100)
    # d loss / d h =  2 * (y_pred - y) * w2
    w1_grad = x.T@h_grad # (1000, 64) * (64, 100) = (1000, 100)
    
    # update parameters
    w1 -= learning_rate * w1_grad
    w2 -= learning_rate * w2_grad

0: loss = 25190348.0
1: loss = 18913532.0
2: loss = 16520530.0
3: loss = 15477824.0
4: loss = 14555248.0
5: loss = 13137495.0
6: loss = 11120706.0
7: loss = 8763305.0
8: loss = 6504405.0
9: loss = 4612281.5
10: loss = 3197467.75
11: loss = 2208325.75
12: loss = 1544545.25
13: loss = 1106569.0
14: loss = 817231.5
15: loss = 623425.9375
16: loss = 490594.25
17: loss = 396976.9375
18: loss = 328826.4375
19: loss = 277561.3125
20: loss = 237781.4375
21: loss = 206080.96875
22: loss = 180245.0
23: loss = 158765.765625
24: loss = 140644.90625
25: loss = 125173.875
26: loss = 111869.390625
27: loss = 100325.265625
28: loss = 90227.8671875
29: loss = 81352.1015625
30: loss = 73515.3828125
31: loss = 66583.5546875
32: loss = 60419.18359375
33: loss = 54922.2890625
34: loss = 50012.57421875
35: loss = 45617.9296875
36: loss = 41671.6953125
37: loss = 38121.78125
38: loss = 34927.96875
39: loss = 32044.8046875
40: loss = 29436.328125
41: loss = 27080.15625
42: loss = 24941.12109375
43: loss = 229

359: loss = 0.0186220221221447
360: loss = 0.01793595962226391
361: loss = 0.017267413437366486
362: loss = 0.01662105694413185
363: loss = 0.01600056327879429
364: loss = 0.015405955724418163
365: loss = 0.014825332909822464
366: loss = 0.01427861861884594
367: loss = 0.013743785209953785
368: loss = 0.013236518949270248
369: loss = 0.012755590490996838
370: loss = 0.012280402705073357
371: loss = 0.011827772483229637
372: loss = 0.01139798667281866
373: loss = 0.010981785133481026
374: loss = 0.010572323575615883
375: loss = 0.010179922915995121
376: loss = 0.009812711738049984
377: loss = 0.009452859871089458
378: loss = 0.009104971773922443
379: loss = 0.00876955222338438
380: loss = 0.00845559872686863
381: loss = 0.008148040622472763
382: loss = 0.007856739684939384
383: loss = 0.007578581105917692
384: loss = 0.007299136370420456
385: loss = 0.007039783988147974
386: loss = 0.006780785042792559
387: loss = 0.006541406735777855
388: loss = 0.006308604031801224
389: loss = 0.00608

### 1.3 Autograd

In [111]:
x=torch.randn(N, D_IN)
y=torch.randn(N, D_OUT)
w1=torch.randn(D_IN, H, requires_grad=True)
w2=torch.randn(H, D_OUT, requires_grad=True)

for epoch in range(NUM_EPOCHS):
    # fowward pass
    y_pred = F.relu(x@w1)@w2
    loss = (y_pred - y).pow(2).sum()
    print(f"{epoch}: loss = {loss.item()}")
    
    # calculate gradients
    loss.backward()
    
    # update parameters
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0: loss = 33568252.0
1: loss = 32938532.0
2: loss = 36839896.0
3: loss = 38404080.0
4: loss = 32796082.0
5: loss = 21467252.0
6: loss = 11210849.0
7: loss = 5359230.0
8: loss = 2764205.5
9: loss = 1680146.875
10: loss = 1189829.0
11: loss = 928319.375
12: loss = 762477.375
13: loss = 642991.6875
14: loss = 550300.6875
15: loss = 475091.375
16: loss = 412737.4375
17: loss = 360368.90625
18: loss = 315947.90625
19: loss = 278006.09375
20: loss = 245425.703125
21: loss = 217308.65625
22: loss = 192968.046875
23: loss = 171802.125
24: loss = 153327.0625
25: loss = 137133.375
26: loss = 122914.1328125
27: loss = 110412.78125
28: loss = 99362.609375
29: loss = 89573.96875
30: loss = 80883.3125
31: loss = 73154.0
32: loss = 66262.671875
33: loss = 60102.44921875
34: loss = 54591.0859375
35: loss = 49650.015625
36: loss = 45209.90625
37: loss = 41220.62890625
38: loss = 37627.390625
39: loss = 34387.2265625
40: loss = 31458.4140625
41: loss = 28811.703125
42: loss = 26413.6875
43: loss = 24239

414: loss = 0.0006368233589455485
415: loss = 0.0006202783551998436
416: loss = 0.0006040924345143139
417: loss = 0.0005884732818230987
418: loss = 0.0005716517916880548
419: loss = 0.000557831022888422
420: loss = 0.0005435434868559241
421: loss = 0.0005301192868500948
422: loss = 0.0005162448505870998
423: loss = 0.0005040462128818035
424: loss = 0.0004923064261674881
425: loss = 0.0004801856121048331
426: loss = 0.0004671549249906093
427: loss = 0.0004555857158266008
428: loss = 0.00044437526958063245
429: loss = 0.0004334961995482445
430: loss = 0.00042416705400682986
431: loss = 0.0004126159765291959
432: loss = 0.00040304329013451934
433: loss = 0.00039356175693683326
434: loss = 0.00038408476393669844
435: loss = 0.0003751155163627118
436: loss = 0.0003667112614493817
437: loss = 0.00035894179018214345
438: loss = 0.00035028214915655553
439: loss = 0.0003430218785069883
440: loss = 0.0003346037701703608
441: loss = 0.0003278341901022941
442: loss = 0.0003195116005372256
443: los

### 1.4 nn

In [117]:
x=torch.randn(N, D_IN)
y=torch.randn(N, D_OUT)

model = nn.Sequential(
    nn.Linear(D_IN, H),
    nn.ReLU(),
    nn.Linear(H, D_OUT)
)

loss_fn = nn.MSELoss(reduction='sum')

for epoch in range(NUM_EPOCHS):
    # fowward pass
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    print(f"{epoch}: loss = {loss.item()}")
    
    # calculate gradients
    model.zero_grad()
    loss.backward()
    
    # update parameters
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0: loss = 711.8518676757812
1: loss = 711.3204956054688
2: loss = 710.790283203125
3: loss = 710.261474609375
4: loss = 709.733642578125
5: loss = 709.206787109375
6: loss = 708.6809692382812
7: loss = 708.1553955078125
8: loss = 707.6304321289062
9: loss = 707.1058959960938
10: loss = 706.5819702148438
11: loss = 706.05859375
12: loss = 705.5360717773438
13: loss = 705.0145874023438
14: loss = 704.49365234375
15: loss = 703.973388671875
16: loss = 703.45361328125
17: loss = 702.9344482421875
18: loss = 702.4159545898438
19: loss = 701.898193359375
20: loss = 701.3818359375
21: loss = 700.8661499023438
22: loss = 700.3511352539062
23: loss = 699.8365478515625
24: loss = 699.3223266601562
25: loss = 698.8087158203125
26: loss = 698.2951049804688
27: loss = 697.7821044921875
28: loss = 697.2693481445312
29: loss = 696.7571411132812
30: loss = 696.2454223632812
31: loss = 695.7342529296875
32: loss = 695.2235717773438
33: loss = 694.71337890625
34: loss = 694.2037963867188
35: loss = 693.

422: loss = 536.8927001953125
423: loss = 536.5682983398438
424: loss = 536.2445068359375
425: loss = 535.9215087890625
426: loss = 535.5989990234375
427: loss = 535.27685546875
428: loss = 534.9550170898438
429: loss = 534.6326904296875
430: loss = 534.310791015625
431: loss = 533.9890747070312
432: loss = 533.6679077148438
433: loss = 533.3469848632812
434: loss = 533.0259399414062
435: loss = 532.705322265625
436: loss = 532.3848876953125
437: loss = 532.0648803710938
438: loss = 531.7451782226562
439: loss = 531.4257202148438
440: loss = 531.1063842773438
441: loss = 530.7871704101562
442: loss = 530.4680786132812
443: loss = 530.1490478515625
444: loss = 529.8305053710938
445: loss = 529.5125732421875
446: loss = 529.194580078125
447: loss = 528.8783569335938
448: loss = 528.5626831054688
449: loss = 528.2474365234375
450: loss = 527.932861328125
451: loss = 527.6181640625
452: loss = 527.3037109375
453: loss = 526.98974609375
454: loss = 526.6759033203125
455: loss = 526.36273193

### 1.5: optim

In [119]:
x=torch.randn(N, D_IN)
y=torch.randn(N, D_OUT)

model = nn.Sequential(
    nn.Linear(D_IN, H),
    nn.ReLU(),
    nn.Linear(H, D_OUT)
)

loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(NUM_EPOCHS):
    # fowward pass
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    print(f"{epoch}: loss = {loss.item()}")
    
    # calculate gradients
    optimizer.zero_grad()
    loss.backward()
    
    # update parameters
    optimizer.step()

0: loss = 630.9755249023438
1: loss = 630.805908203125
2: loss = 630.636474609375
3: loss = 630.4671630859375
4: loss = 630.2977905273438
5: loss = 630.1287231445312
6: loss = 629.959716796875
7: loss = 629.790771484375
8: loss = 629.6217041015625
9: loss = 629.4527587890625
10: loss = 629.2838134765625
11: loss = 629.1148681640625
12: loss = 628.9459838867188
13: loss = 628.777099609375
14: loss = 628.6083374023438
15: loss = 628.4395751953125
16: loss = 628.2708740234375
17: loss = 628.1021728515625
18: loss = 627.9335327148438
19: loss = 627.7650756835938
20: loss = 627.5967407226562
21: loss = 627.4283447265625
22: loss = 627.2601318359375
23: loss = 627.0918579101562
24: loss = 626.9237670898438
25: loss = 626.7559204101562
26: loss = 626.5880126953125
27: loss = 626.4201049804688
28: loss = 626.2523193359375
29: loss = 626.084716796875
30: loss = 625.9171142578125
31: loss = 625.7496337890625
32: loss = 625.58203125
33: loss = 625.41455078125
34: loss = 625.2470703125
35: loss = 

351: loss = 574.6937255859375
352: loss = 574.542236328125
353: loss = 574.3907470703125
354: loss = 574.2393798828125
355: loss = 574.0880126953125
356: loss = 573.9368286132812
357: loss = 573.785400390625
358: loss = 573.6340942382812
359: loss = 573.4830322265625
360: loss = 573.3321533203125
361: loss = 573.1810913085938
362: loss = 573.0301513671875
363: loss = 572.8790893554688
364: loss = 572.7281494140625
365: loss = 572.5772705078125
366: loss = 572.4264526367188
367: loss = 572.2756958007812
368: loss = 572.1248168945312
369: loss = 571.97412109375
370: loss = 571.823486328125
371: loss = 571.673095703125
372: loss = 571.5225830078125
373: loss = 571.3721923828125
374: loss = 571.2218017578125
375: loss = 571.0714721679688
376: loss = 570.9212646484375
377: loss = 570.7711181640625
378: loss = 570.620849609375
379: loss = 570.4706420898438
380: loss = 570.320556640625
381: loss = 570.1703491210938
382: loss = 570.020263671875
383: loss = 569.8704223632812
384: loss = 569.720

### 1.6 class

In [122]:
x=torch.randn(N, D_IN)
y=torch.randn(N, D_OUT)

class Model(torch.nn.Module):
    def __init__(self, D_IN, H, D_OUT):
        super().__init__()
        self.linear1 = torch.nn.Linear(D_IN, H)
        self.linear2 = torch.nn.Linear(H, D_OUT)

    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
model = Model(D_IN, H, D_OUT)
loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(NUM_EPOCHS):
    # fowward pass
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    print(f"{epoch}: loss = {loss.item()}")
    
    # calculate gradients
    optimizer.zero_grad()
    loss.backward()
    
    # update parameters
    optimizer.step()

0: loss = 679.6824951171875
1: loss = 679.499755859375
2: loss = 679.317138671875
3: loss = 679.134521484375
4: loss = 678.9519653320312
5: loss = 678.7694702148438
6: loss = 678.5872192382812
7: loss = 678.405029296875
8: loss = 678.2227783203125
9: loss = 678.0405883789062
10: loss = 677.8583374023438
11: loss = 677.67626953125
12: loss = 677.494140625
13: loss = 677.3120727539062
14: loss = 677.1300048828125
15: loss = 676.9480590820312
16: loss = 676.76611328125
17: loss = 676.5841674804688
18: loss = 676.4024658203125
19: loss = 676.2205810546875
20: loss = 676.038818359375
21: loss = 675.8571166992188
22: loss = 675.6754760742188
23: loss = 675.4939575195312
24: loss = 675.3124389648438
25: loss = 675.1309204101562
26: loss = 674.949462890625
27: loss = 674.76806640625
28: loss = 674.5867919921875
29: loss = 674.4054565429688
30: loss = 674.2242431640625
31: loss = 674.0431518554688
32: loss = 673.8621826171875
33: loss = 673.6812133789062
34: loss = 673.5003662109375
35: loss = 

390: loss = 612.8477172851562
391: loss = 612.6868896484375
392: loss = 612.526123046875
393: loss = 612.3653564453125
394: loss = 612.2046508789062
395: loss = 612.0441284179688
396: loss = 611.8836669921875
397: loss = 611.7232055664062
398: loss = 611.5626220703125
399: loss = 611.4025268554688
400: loss = 611.2423706054688
401: loss = 611.0820922851562
402: loss = 610.9219970703125
403: loss = 610.7618408203125
404: loss = 610.6016235351562
405: loss = 610.441650390625
406: loss = 610.2816772460938
407: loss = 610.1218872070312
408: loss = 609.962158203125
409: loss = 609.802490234375
410: loss = 609.642822265625
411: loss = 609.4832763671875
412: loss = 609.3236694335938
413: loss = 609.1641235351562
414: loss = 609.004638671875
415: loss = 608.84521484375
416: loss = 608.685791015625
417: loss = 608.5263671875
418: loss = 608.366943359375
419: loss = 608.2078857421875
420: loss = 608.048583984375
421: loss = 607.8895263671875
422: loss = 607.7303466796875
423: loss = 607.57128906

## 2. fizzbuzz game

In [49]:
torch.cuda.empty_cache()

In [43]:
def fizzbuzz_number(i):
    if i%15==0: return 3
    elif i%5==0: return 2
    elif i%3==0: return 1
    else: return 0

def fizzbuzz_nts(i, p):
    return [str(i),'fizz', 'buzz', 'fizzbuzz'][p]

NUM_DIGIT = 14

def binary(i, digit=NUM_DIGIT):
    return np.array([i>>d &1 for d in range(digit)])

trX = torch.Tensor([binary(i) for i in range(101, 2**NUM_DIGIT)]).cuda()
trY = torch.LongTensor([fizzbuzz_number(i) for i in range(101, 2**NUM_DIGIT)]).cuda()

In [44]:
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGIT, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4),
    torch.nn.Sigmoid()
)

if USE_CUDA:
    model = model.cuda()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

In [45]:
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Find loss on training data
    with torch.no_grad():
        loss = loss_fn(model(trX), trY)
        print('Epoch:', epoch, 'Loss:', loss.item())
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

Epoch: 0 Loss: 1.3966401815414429
Epoch: 1 Loss: 1.3918284177780151
Epoch: 2 Loss: 1.387069582939148
Epoch: 3 Loss: 1.382372260093689
Epoch: 4 Loss: 1.3777461051940918
Epoch: 5 Loss: 1.3731940984725952
Epoch: 6 Loss: 1.368726372718811
Epoch: 7 Loss: 1.3643498420715332
Epoch: 8 Loss: 1.360068917274475
Epoch: 9 Loss: 1.355888843536377
Epoch: 10 Loss: 1.3518143892288208
Epoch: 11 Loss: 1.3478491306304932
Epoch: 12 Loss: 1.3439946174621582
Epoch: 13 Loss: 1.3402533531188965
Epoch: 14 Loss: 1.3366261720657349
Epoch: 15 Loss: 1.3331143856048584
Epoch: 16 Loss: 1.3297159671783447
Epoch: 17 Loss: 1.326429009437561
Epoch: 18 Loss: 1.3232545852661133
Epoch: 19 Loss: 1.3201913833618164
Epoch: 20 Loss: 1.3172355890274048
Epoch: 21 Loss: 1.3143830299377441
Epoch: 22 Loss: 1.3116326332092285
Epoch: 23 Loss: 1.308980941772461
Epoch: 24 Loss: 1.306424856185913
Epoch: 25 Loss: 1.3039604425430298
Epoch: 26 Loss: 1.3015859127044678
Epoch: 27 Loss: 1.2992968559265137
Epoch: 28 Loss: 1.2970900535583496
Epo

Epoch: 235 Loss: 1.2104278802871704
Epoch: 236 Loss: 1.210349440574646
Epoch: 237 Loss: 1.2102711200714111
Epoch: 238 Loss: 1.2101929187774658
Epoch: 239 Loss: 1.2101163864135742
Epoch: 240 Loss: 1.210039734840393
Epoch: 241 Loss: 1.2099636793136597
Epoch: 242 Loss: 1.2098883390426636
Epoch: 243 Loss: 1.2098134756088257
Epoch: 244 Loss: 1.2097389698028564
Epoch: 245 Loss: 1.2096654176712036
Epoch: 246 Loss: 1.209592342376709
Epoch: 247 Loss: 1.2095197439193726
Epoch: 248 Loss: 1.2094476222991943
Epoch: 249 Loss: 1.2093756198883057
Epoch: 250 Loss: 1.209304690361023
Epoch: 251 Loss: 1.2092342376708984
Epoch: 252 Loss: 1.209164023399353
Epoch: 253 Loss: 1.2090944051742554
Epoch: 254 Loss: 1.209025263786316
Epoch: 255 Loss: 1.2089563608169556
Epoch: 256 Loss: 1.2088887691497803
Epoch: 257 Loss: 1.2088210582733154
Epoch: 258 Loss: 1.208753228187561
Epoch: 259 Loss: 1.2086867094039917
Epoch: 260 Loss: 1.2086204290390015
Epoch: 261 Loss: 1.2085551023483276
Epoch: 262 Loss: 1.2084892988204956

Epoch: 465 Loss: 1.2008233070373535
Epoch: 466 Loss: 1.2008017301559448
Epoch: 467 Loss: 1.2007802724838257
Epoch: 468 Loss: 1.200758934020996
Epoch: 469 Loss: 1.200737714767456
Epoch: 470 Loss: 1.200716257095337
Epoch: 471 Loss: 1.2006956338882446
Epoch: 472 Loss: 1.2006745338439941
Epoch: 473 Loss: 1.2006535530090332
Epoch: 474 Loss: 1.2006325721740723
Epoch: 475 Loss: 1.200612187385559
Epoch: 476 Loss: 1.2005913257598877
Epoch: 477 Loss: 1.2005707025527954
Epoch: 478 Loss: 1.2005505561828613
Epoch: 479 Loss: 1.2005298137664795
Epoch: 480 Loss: 1.2005095481872559
Epoch: 481 Loss: 1.2004896402359009
Epoch: 482 Loss: 1.2004694938659668
Epoch: 483 Loss: 1.2004499435424805
Epoch: 484 Loss: 1.2004295587539673
Epoch: 485 Loss: 1.2004095315933228
Epoch: 486 Loss: 1.2003902196884155
Epoch: 487 Loss: 1.2003700733184814
Epoch: 488 Loss: 1.2003508806228638
Epoch: 489 Loss: 1.200331211090088
Epoch: 490 Loss: 1.2003117799758911
Epoch: 491 Loss: 1.2002924680709839
Epoch: 492 Loss: 1.20027351379394

Epoch: 699 Loss: 1.1976350545883179
Epoch: 700 Loss: 1.1976279020309448
Epoch: 701 Loss: 1.1976194381713867
Epoch: 702 Loss: 1.1976120471954346
Epoch: 703 Loss: 1.197603464126587
Epoch: 704 Loss: 1.1975966691970825
Epoch: 705 Loss: 1.1975886821746826
Epoch: 706 Loss: 1.1975808143615723
Epoch: 707 Loss: 1.1975730657577515
Epoch: 708 Loss: 1.1975656747817993
Epoch: 709 Loss: 1.197557806968689
Epoch: 710 Loss: 1.1975507736206055
Epoch: 711 Loss: 1.1975429058074951
Epoch: 712 Loss: 1.1975349187850952
Epoch: 713 Loss: 1.19752836227417
Epoch: 714 Loss: 1.1975207328796387
Epoch: 715 Loss: 1.1975131034851074
Epoch: 716 Loss: 1.1975055932998657
Epoch: 717 Loss: 1.197498083114624
Epoch: 718 Loss: 1.1974903345108032
Epoch: 719 Loss: 1.197484016418457
Epoch: 720 Loss: 1.1974761486053467
Epoch: 721 Loss: 1.1974689960479736
Epoch: 722 Loss: 1.1974622011184692
Epoch: 723 Loss: 1.1974551677703857
Epoch: 724 Loss: 1.1974475383758545
Epoch: 725 Loss: 1.197440505027771
Epoch: 726 Loss: 1.1974331140518188

Epoch: 929 Loss: 1.1965711116790771
Epoch: 930 Loss: 1.1965696811676025
Epoch: 931 Loss: 1.196567416191101
Epoch: 932 Loss: 1.1965651512145996
Epoch: 933 Loss: 1.1965640783309937
Epoch: 934 Loss: 1.1965618133544922
Epoch: 935 Loss: 1.196559190750122
Epoch: 936 Loss: 1.1965581178665161
Epoch: 937 Loss: 1.1965558528900146
Epoch: 938 Loss: 1.1965540647506714
Epoch: 939 Loss: 1.1965521574020386
Epoch: 940 Loss: 1.1965501308441162
Epoch: 941 Loss: 1.1965489387512207
Epoch: 942 Loss: 1.196547031402588
Epoch: 943 Loss: 1.1965460777282715
Epoch: 944 Loss: 1.1965439319610596
Epoch: 945 Loss: 1.196541666984558
Epoch: 946 Loss: 1.1965408325195312
Epoch: 947 Loss: 1.1965385675430298
Epoch: 948 Loss: 1.1965364217758179
Epoch: 949 Loss: 1.1965357065200806
Epoch: 950 Loss: 1.1965337991714478
Epoch: 951 Loss: 1.1965316534042358
Epoch: 952 Loss: 1.196530818939209
Epoch: 953 Loss: 1.1965287923812866
Epoch: 954 Loss: 1.1965270042419434
Epoch: 955 Loss: 1.1965259313583374
Epoch: 956 Loss: 1.19652402400970

...

Epoch: 9023 Loss: 1.2168489694595337
Epoch: 9024 Loss: 1.2168540954589844
Epoch: 9025 Loss: 1.2168593406677246
Epoch: 9026 Loss: 1.216863989830017
Epoch: 9027 Loss: 1.2168693542480469
Epoch: 9028 Loss: 1.216874599456787
Epoch: 9029 Loss: 1.2168794870376587
Epoch: 9030 Loss: 1.2168848514556885
Epoch: 9031 Loss: 1.2168899774551392
Epoch: 9032 Loss: 1.2168948650360107
Epoch: 9033 Loss: 1.2169002294540405
Epoch: 9034 Loss: 1.216905117034912
Epoch: 9035 Loss: 1.2169102430343628
Epoch: 9036 Loss: 1.216915249824524
Epoch: 9037 Loss: 1.2169201374053955
Epoch: 9038 Loss: 1.216925859451294
Epoch: 9039 Loss: 1.2169303894042969
Epoch: 9040 Loss: 1.216935396194458
Epoch: 9041 Loss: 1.2169406414031982
Epoch: 9042 Loss: 1.216945767402649
Epoch: 9043 Loss: 1.2169508934020996
Epoch: 9044 Loss: 1.2169562578201294
Epoch: 9045 Loss: 1.2169610261917114
Epoch: 9046 Loss: 1.216966152191162
Epoch: 9047 Loss: 1.2169713973999023
Epoch: 9048 Loss: 1.2169764041900635
Epoch: 9049 Loss: 1.216981053352356
Epoch: 905

Epoch: 9248 Loss: 1.2180005311965942
Epoch: 9249 Loss: 1.2180054187774658
Epoch: 9250 Loss: 1.2180103063583374
Epoch: 9251 Loss: 1.218015432357788
Epoch: 9252 Loss: 1.2180207967758179
Epoch: 9253 Loss: 1.2180252075195312
Epoch: 9254 Loss: 1.2180308103561401
Epoch: 9255 Loss: 1.2180354595184326
Epoch: 9256 Loss: 1.218040108680725
Epoch: 9257 Loss: 1.2180454730987549
Epoch: 9258 Loss: 1.2180498838424683
Epoch: 9259 Loss: 1.2180548906326294
Epoch: 9260 Loss: 1.2180598974227905
Epoch: 9261 Loss: 1.2180652618408203
Epoch: 9262 Loss: 1.218070149421692
Epoch: 9263 Loss: 1.2180747985839844
Epoch: 9264 Loss: 1.2180801630020142
Epoch: 9265 Loss: 1.218084692955017
Epoch: 9266 Loss: 1.2180896997451782
Epoch: 9267 Loss: 1.2180943489074707
Epoch: 9268 Loss: 1.218099594116211
Epoch: 9269 Loss: 1.218104362487793
Epoch: 9270 Loss: 1.2181090116500854
Epoch: 9271 Loss: 1.218113899230957
Epoch: 9272 Loss: 1.2181180715560913
Epoch: 9273 Loss: 1.2181227207183838
Epoch: 9274 Loss: 1.2181278467178345
Epoch: 9

Epoch: 9473 Loss: 1.2190035581588745
Epoch: 9474 Loss: 1.219007134437561
Epoch: 9475 Loss: 1.2190115451812744
Epoch: 9476 Loss: 1.2190155982971191
Epoch: 9477 Loss: 1.219019889831543
Epoch: 9478 Loss: 1.2190238237380981
Epoch: 9479 Loss: 1.2190279960632324
Epoch: 9480 Loss: 1.2190321683883667
Epoch: 9481 Loss: 1.219036340713501
Epoch: 9482 Loss: 1.219040036201477
Epoch: 9483 Loss: 1.2190443277359009
Epoch: 9484 Loss: 1.2190489768981934
Epoch: 9485 Loss: 1.2190526723861694
Epoch: 9486 Loss: 1.2190567255020142
Epoch: 9487 Loss: 1.2190607786178589
Epoch: 9488 Loss: 1.2190654277801514
Epoch: 9489 Loss: 1.219069004058838
Epoch: 9490 Loss: 1.219072937965393
Epoch: 9491 Loss: 1.2190771102905273
Epoch: 9492 Loss: 1.2190816402435303
Epoch: 9493 Loss: 1.2190852165222168
Epoch: 9494 Loss: 1.219089388847351
Epoch: 9495 Loss: 1.219093680381775
Epoch: 9496 Loss: 1.2190979719161987
Epoch: 9497 Loss: 1.2191015481948853
Epoch: 9498 Loss: 1.21910560131073
Epoch: 9499 Loss: 1.2191097736358643
Epoch: 9500

Epoch: 9699 Loss: 1.2198301553726196
Epoch: 9700 Loss: 1.2198328971862793
Epoch: 9701 Loss: 1.2198357582092285
Epoch: 9702 Loss: 1.2198394536972046
Epoch: 9703 Loss: 1.2198419570922852
Epoch: 9704 Loss: 1.2198448181152344
Epoch: 9705 Loss: 1.2198486328125
Epoch: 9706 Loss: 1.2198514938354492
Epoch: 9707 Loss: 1.219854474067688
Epoch: 9708 Loss: 1.2198582887649536
Epoch: 9709 Loss: 1.219860553741455
Epoch: 9710 Loss: 1.2198641300201416
Epoch: 9711 Loss: 1.2198671102523804
Epoch: 9712 Loss: 1.219869613647461
Epoch: 9713 Loss: 1.219873070716858
Epoch: 9714 Loss: 1.2198761701583862
Epoch: 9715 Loss: 1.2198795080184937
Epoch: 9716 Loss: 1.2198824882507324
Epoch: 9717 Loss: 1.2198853492736816
Epoch: 9718 Loss: 1.2198880910873413
Epoch: 9719 Loss: 1.2198916673660278
Epoch: 9720 Loss: 1.2198947668075562
Epoch: 9721 Loss: 1.2198970317840576
Epoch: 9722 Loss: 1.2199008464813232
Epoch: 9723 Loss: 1.219903826713562
Epoch: 9724 Loss: 1.2199064493179321
Epoch: 9725 Loss: 1.2199101448059082
Epoch: 97

Epoch: 9924 Loss: 1.220440149307251
Epoch: 9925 Loss: 1.2204421758651733
Epoch: 9926 Loss: 1.220445156097412
Epoch: 9927 Loss: 1.2204474210739136
Epoch: 9928 Loss: 1.220449686050415
Epoch: 9929 Loss: 1.2204519510269165
Epoch: 9930 Loss: 1.2204539775848389
Epoch: 9931 Loss: 1.2204561233520508
Epoch: 9932 Loss: 1.2204585075378418
Epoch: 9933 Loss: 1.220461130142212
Epoch: 9934 Loss: 1.220463514328003
Epoch: 9935 Loss: 1.2204655408859253
Epoch: 9936 Loss: 1.2204680442810059
Epoch: 9937 Loss: 1.2204703092575073
Epoch: 9938 Loss: 1.2204724550247192
Epoch: 9939 Loss: 1.2204746007919312
Epoch: 9940 Loss: 1.2204774618148804
Epoch: 9941 Loss: 1.2204792499542236
Epoch: 9942 Loss: 1.2204817533493042
Epoch: 9943 Loss: 1.2204843759536743
Epoch: 9944 Loss: 1.2204864025115967
Epoch: 9945 Loss: 1.2204889059066772
Epoch: 9946 Loss: 1.2204906940460205
Epoch: 9947 Loss: 1.2204928398132324
Epoch: 9948 Loss: 1.2204952239990234
Epoch: 9949 Loss: 1.2204978466033936
Epoch: 9950 Loss: 1.220499873161316
Epoch: 

In [38]:
testX = torch.Tensor([binary(i, NUM_DIGIT) for i in range(1, 101)]).cuda()
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizzbuzz_nts(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', '5', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', '24', '25', '26', 'fizz', '28', '29', 'fizzbuzz', '31', 'buzz', 'fizz', 'fizz', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', '70', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', '95', 'fizz', '97', '98', 'fizz', 'buzz']


In [39]:
x=torch.randn(100,10)

In [40]:
print(np.sum(testY.cpu().max(1)[1].numpy() == np.array([fizzbuzz_number(i) for i in range(1,101)])))
testY.cpu().max(1)[1].numpy() == np.array([fizzbuzz_number(i) for i in range(1,101)])

93


array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True])